## Vehicle Speed Estimation
Estimating speed of vehicle with YOLOv5 and ByteTracker

## Installing reqruied packages & environment setup

In [1]:
import os
HOME = os.getcwd()

In [ ]:
# Clones the YOLOv5 repository from GitHub using Git
!git clone https://github.com/ultralytics/yolov5  # clone

# Changes the current working directory to yolov5
%cd yolov5

# Installs required Python packages using pip
!pip install -r requirements.txt  # install

In [ ]:
# Changes the current working directory to HOME
%cd {HOME}

# Clones the ByteTrack repository from GitHub
!git clone https://github.com/ifzhang/ByteTrack.git

# Installs the required Python packages using pip3
!cd ByteTrack && pip3 install -r requirements.txt

# Runs the setup.py script using python3 to install the ByteTrack package
!cd ByteTrack && python3 setup.py develop

# Installs additional Python packages using pip
!pip install cython_bbox loguru lap

# Installs the onemetric package using pip, with the --quiet flag to suppress output
!pip install onemetric --quiet


In [ ]:
# Imports the 'sys' module, which provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter.
import sys

# Appends a path to the system path, which is a list of directories that the interpreter will search for modules when importing them.
# The path that is appended is constructed using an f-string that includes the value of HOME.
sys.path.append(f"{HOME}/ByteTrack")

## Importing requirec packages


In [ ]:
import torch
import cv2
from dataclasses import dataclass
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
import time
import numpy as np

## Accessories & Initializing...

In [ ]:
# Define the arguments for BYTETracker
@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

# Define a function to calculate the speed of a vehicle
def calculate_speed(previous_centroid, current_centroid, previous_frame_time, current_frame_time):
    # Calculate the distance traveled by the vehicle
    distance = np.linalg.norm(current_centroid - previous_centroid)

    # Calculate the time taken by the vehicle to travel the distance
    time_delta = current_frame_time - previous_frame_time

    # Calculate the speed of the vehicle in pixels per second
    speed = distance / time_delta

    # Return the speed in kilometers per hour
    return speed * 3.6
  
#Initialize previous frame time 
previous_frame_time = None
# Initiate the BYTETracker object
tracker = BYTETracker(BYTETrackerArgs)

# Define the vehicle classes to be tracked
vehicle_classes = ['car', 'bus', 'truck']

# Get the names of the classes from the YOLOv5 model
vehicle_names = model.names

# Initialize a dictionary to store the bounding boxes for each track ID in the previous frame
prevoius_online_tracker  = {}

## Loading model

In [ ]:
# Load the YOLOv5 model
model = torch.hub.load("ultralytics/yolov5", "yolov5s")  # or yolov5n - yolov5x6, custom

nmodel = torch.hub.load("ultralytics/yolov5", 'custom', "/Users/auk/Downloads/DataProjects/speed_estimation/best.pt")  # or yolov5n - yolov5x6, custom

## Input data

In [ ]:
# Open the video file for processing
cap = cv2.VideoCapture('/Users/auk/Downloads/DataProjects/speed_estimation/b.mp4')

## Input Processing & Speed estimation

In [ ]:
# Loop through each frame of the video
while True:
    # Read the frame
    ret, frame = cap.read()

    if not ret:
        break

    # Perform object detection with YOLOv5
    results = model(frame)

    # Update the tracker with the detection results for the vehicle classes
    for det in results.pred[0]:
        class_idx = int(det[-1])
        conf = float(det[-2])
        if vehicle_names[class_idx] in vehicle_classes and conf > 0.7:
            online_tracker= tracker.update(results.pred[0], img_info=frame.shape, img_size=frame.shape)

    # Draw the bounding boxes and track IDs on the frame
    for obj in online_tracker:
        x1, y1, x2, y2 = map(int, obj.tlbr)
        current_frame_time = time.time()

        if previous_frame_time == None:
            break
        if obj.track_id in prevoius_online_tracker.keys():
            px1, py1, px2, py2 = prevoius_online_tracker[obj.track_id]
            previous_centroid = np.array([(px1+px2)/2, (py1+py2)/2])
            current_centroid = np.array([(x1+x2)/2, (y1+y2)/2])
            speed = calculate_speed(previous_centroid, current_centroid, previous_frame_time, current_frame_time)
            if speed > 0:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
                cv2.putText(frame, f'Speed: {speed:.2f} km/h', (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)

        prevoius_online_tracker[obj.track_id] = [x1, y1, x2, y2]
        

    previous_frame_time = current_frame_time

    # Display the processed frame
    cv2.imshow('Speed Estimator', frame)

    # Exit the loop if the 's' key is pressed
    if cv2.waitKey(1) == ord('s'):
        break

# Release the resources
cap.release()
cv2.destroyAllWindows()
